In [ ]:
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv("../credentials.env")

client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT","").strip(),
    api_key        = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version    = os.getenv("OPENAI_API_VERSION")
)

encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
deployment_name = os.getenv('DEPLOYMENT_NAME')

# A Few Shot Learning

In [ ]:
# Zero-shot classification
system_prompt ="""Predict up to 5 emojis as a response to a text chat message. The output 
should only include emojis.

input: The new visual design is blowing my mind 🤯
output: ➕,💘, ❤‍🔥

input: Well that looks great regardless
output: ❤️,🪄

input: Unfortunately this won't work
output: 💔,😔

input: sounds good, I'll look into that
output: 🙏,👍

input: 10hr cut of jeff goldblum laughing URL
output: 😂,💀,⚰️
"""
# 문자 채팅 메시지에 대한 응답으로 최대 5개의 이모티콘을 예측하십시오. 출력에는 이모티콘만 포함되어야 합니다.

# 입력 : 새로운 시각적 디자인이 내 마음을 설레게 합니다
# 출력 : ➕,💘, ❤‍🔥

# 입력 : 모든 상관없이 멋져 보입니다
# 출력 : ❤️,🪄

# 입력 : 안타깝게도 그건 안돼요.
# 출력 : 💔,😔

# 입력 : 좋습니다. 한번 해볼게요.
# 출력 : 🙏,👍

# 입력 : 10시간 삭제 jeff goldblum의 웃음 URL 
# 출력 : 😂,💀,⚰️

user_prompt = "The new user interface is amazing!" # 새로운 사용자 인터페이스는 놀랍습니다!
response = client.chat.completions.create(
    model=deployment_name,
    messages = [{"role":"system", "content":system_prompt},
                {"role":"user","content": user_prompt,}])
print(response.choices[0].message.content)

# Chain of Thought
실험 결과는 한 개의 프롬프트 템플릿을 사용하여 Zero-shot-CoT가 다양한 벤치마크 추론 작업에서 Zero-shot LLM 성능을 현저하게 능가함을 보여줍니다. 손으로 작성된 몇 가지 샷 예시 없이도, 예를 들어, 대규모 InstructGPT 모델 (text-davinci-002)을 사용하여 MultiArith에서의 정확도를 17.7%에서 78.7%로, 그리고 GSM8K에서의 정확도를 10.4%에서 40.7%로 향상시킵니다.

Source: [Large Language Models are Zero-Shot Reasoners](https://arxiv.org/abs/2205.11916)

In [ ]:
def call_openai(prompt, temperature=1, max_token=60):
    response = client.chat.completions.create(
        model=deployment_name,
        messages = prompt,
            max_tokens=max_token,
            temperature = temperature
    )
    return response.choices[0].message.content

In [ ]:
# ZERO_SHOT Prompt

system_prompt = """All answers are in korean. Answer the following questions:"""
quiz = """The bakers at the Beverly Hills Bakery baked 200 loaves of bread on Monday morning.
They sold 93 loaves in the morning and 39 loaves in the afternoon.
A grocery store returned 6 unsold loaves.
How many loaves of bread did they have left?"""

# 밑의 질문들을 대답하시오 

# 베벌리힐스 베이커리의 제빵사들은 월요일 아침 200개의 빵을 구웠습니다.
# 아침에 93개, 오후에 39개의 빵을 팔았습니다.
# 한 식료품점에서 팔리지 않은 빵 6개를 반품했습니다.
# 그들은 빵 몇 덩어리를 남겼습니까?

prompt = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": quiz}
]

response_msg = call_openai(prompt, temperature=0, max_token=200)

print(response_msg)

In [ ]:
# FEW-SHOT Learning Prompt

prompt=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "Q: Roger has 5 tennis balss. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does Roger have now?"},
    {"role": "assistant", "content": "A: The answer is 11."},
    {"role": "user", "content": quiz}
]

response_msg = call_openai(prompt, temperature=0, max_token=200)

print(response_msg)

# Commonsense Reasoning

Paper: [Generated Knowledge Prompting for Commonsense Reasoning](https://arxiv.org/abs/2110.08387)


3가지 기여 요인은 다음과 같습니다:

(i) 지식의 품질: 사용된 지식이 얼마나 정확하고 유용한지가 중요합니다. 품질이 높은 지식일수록 결과가 더 좋을 가능성이 있습니다.

(ii) 지식의 양: 성능이 더 많은 지식 문장으로 향상될 수 있습니다. 즉, 지식의 양이 많을수록 성능이 향상될 가능성이 높습니다.

(iii) 추론 중에 지식을 통합하는 전략: 지식을 효과적으로 활용하고 추론 과정에서 통합하는 전략이 중요합니다. 지식을 적절하게 활용하는 전략을 갖추면 성능이 향상될 수 있습니다.

In [ ]:
common_question = """
The player with the lowest score wins.
Is this true or false: Part of golf is trying to get a higher point total than others.
"""
# 가장 낮은 점수를 받은 선수가 이깁니다.
# 진실 혹은 거짓 : 골프는 높은 총점을 얻기 위해 노력하는 것입니다.

prompt = [
    {"role": "system", "content": "You are a helpful assistant. All answers are in korean. "},
    {"role": "user", "content": common_question},
]

response_msg = call_openai(prompt, temperature=0, max_token=100)

print(response_msg)

In [ ]:

common_question = """ 
A tripod is a kind of easel
How many legs does an easel have?
"""

# 삼각대는 이젤의 한 종류입니다. 
# 이젤은 다리가 몇 개 입까요?

prompt = [
    {"role": "system", "content": "You are a helpful assistant. All answers are in korean."},
    {"role": "user", "content": common_question},
]

response_msg = call_openai(prompt, temperature=0, max_token=100)

print(response_msg)

In [ ]:
common_question = """Question: Part of golf is trying to get a higher point total than others. Yes or No?
Knowledge: The objective of golf is to play a set of holes in the least number of strokes. A round of golf typically consists of 18 holes. Each hole is played once in the round on a standard golf course. Each stroke is counted as one point, and the total number of strokes is used to determine the winner of the game.
Explain and Answer: 
"""
# 질문: 골프의 일부분은 다른 것들보다 더 높은 총점을 얻기 위해 노력하는 것입니다. 예 또는 아니오?
# 지식: 골프의 목적은 가장 적은 수의 타수로 한 세트의 홀을 플레이하는 것입니다. 한 라운드의 골프는 일반적으로 18개의 홀로 구성됩니다. 
#       각 홀은 일반적인 골프 코스에서 한 라운드에 한 번 플레이됩니다. 각 타수는 1점으로 계산되고, 총 타수는 게임의 승자를 결정하는 데 사용됩니다
# 설명 및 답변:

prompt=[
    {"role": "system", "content": "You are a helpful assistant. All answers are in korean. "},
    {"role": "user", "content": common_question},
]

response_msg = call_openai(prompt, temperature=0, max_token=200)

print(response_msg)